In [1]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from random import randint
from random import uniform
from time import sleep
import os

In [2]:
user_agent_ios = "Mozilla/5.0 (iPhone; U; CPU iPhone OS 3_0 like Mac OS X; en-us) AppleWebKit/528.18 (KHTML, like Gecko) Version/4.0 Mobile/7A341 Safari/528.16"
user_agent_pc = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"

In [3]:
def make_fake_browser(user_agent):  
    profile = webdriver.FirefoxProfile() 
    profile.set_preference("general.useragent.override", user_agent)
    driver = webdriver.Firefox(firefox_profile=profile, firefox_binary=binary, executable_path=r'D:\Projects\AuxTools\geckodriver.exe')
    if "iPhone" in user_agent:
        driver.set_window_size(360,640)
    return driver

In [4]:
binary = FirefoxBinary(r'D:\Projects\AuxTools\Firefox_Portable_80\FirefoxPortable\App\Firefox64\firefox.exe') # путь к firefox
driver = make_fake_browser(user_agent_pc)

In [5]:
products_categories = {
    'debitcards':'Дебетовые карты',
    'creditcards':'Кредитные карты',
    'hypothec':'Ипотека', 
    'autocredits':'Автокредит',
    'credits':'Потребительский кредит',
    'restructing':'Реструктуризация/рефинансирование',
    'deposits':'Вклад',
    'investments':'Инвестиционные продукты/брокерское обслуживание',
    'transfers':'Денежный перевод',
    'remote':'Дистанционное обслуживание физ. лиц',
    'corporate':'Обслуживание юр. лиц',
    'rko':'Расчетно-кассовое обслуживание',
    'acquiring':'Эквайринг',
    'salary_project':'Зарплатный проект',
    'businessdeposits':'Депозит',
    'businesscredits':'Кредитование бизнеса',
    'bank_guarantee':'Банковская гарантия',
    'leasing':'Лизинг',
    'business_other':'Другое',
    'business_remote':'Дистанционное обслуживание юр. лиц'
}

In [ ]:
for key, value in products_categories.items():
    driver.get(f'https://www.banki.ru/services/responses/list/product/{key}/')
    driver.execute_script("window.scrollTo(0, 6000);")
    sleep(randint(1,3))
    count = driver.find_element_by_xpath("//li[contains(@class, 'ui-pagination__item--number')][last()]").text
    sleep(randint(2,4))
    for i in range(int(count)):
        for attempt in range(10):
            sleep(randint(2,4))
            try:
                driver.get(f'https://www.banki.ru/services/responses/list/product/{key}/?page={i+1}')
                content = driver.find_elements_by_xpath("//script[@type = 'application/ld+json']")[1].get_attribute('text')
                with open(f'JSON\\results_bankiru_{key}_{i+1}.json', 'w', encoding='utf8') as outfile:
                    json.dump(content, outfile, indent=4, ensure_ascii=False )
            except Exception as e:
                #print(e)
                print(f'Exception caught ({attempt+1}/{10}). Retrying,,,') 
            break